In [1]:
import requests
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Read in bracket statistics from the Player Index page on the Chip's Bracket site.

Pandas will read in each html table to a df, so we need to take the extra step of identifying the table that contains all of the bracket statistics.

From inspection, we see that the bracket table is the only one with more than a few rows.  We know this pool always has at least 100 submissions so we'll set 100 as our cut-point to pluck out the bracket table. 

In [2]:
url = "http://www.mysideline.com/u/2585/collegebbmens/configurablestandings.htm"
dfs = pd.read_html(url)

for i, df in enumerate(dfs):
    # at least 100 rows (brackets table)
    if dfs[i].shape[0] >= 100:
        bracket_df = df
        break

# preview df
bracket_df.head()

Place Mover/Loser      Name Last,First TotalPoints PossiblePoints  \
  Place Mover/Loser      Name Last,First TotalPoints PossiblePoints   
0  284t      (-121)   1 Berry, Deiondrea          23            153   
1   31t      (+132)  1PEDERSEN,ROBYNALAN          27            163   
2   51t       (+23)   2 Berry, Deiondrea          26            152   
3  185t      (-111)   2 McNamara, justin          24            162   
4  483t       (-35)  2PEDERSEN,ROBYNALAN          19            147   

  Points Gauge PickPct. Final             PickedChamp  
  Points Gauge PickPct.    32 16  8  4  2 PickedChamp  
0        23130      61%    21  1  0  0  0          ??  
1        27136      67%    21  3  0  0  0          ??  
2        26126      67%    22  2  0  0  0          ??  
3        24138      64%    22  1  0  0  0          ??  
4        19128      50%    17  1  0  0  0          ??

We also notice that the first table contains the as of date.

In [3]:
dfs[0]

,0,1,2
0,NaN,Chip's 2021 Tournament As of: 3/21/2021 at 8:2...,NaN


We can parse out the as of date using regex.

In [4]:
# search full string length
pd.options.display.max_colwidth = 200

# regex to parse out the as of date
date_search = re.search(r'(\d+/\d+/\d+ at \d+\:\d+ [A|P]M)', str(dfs[0][1]))
as_of_date = date_search.group(1) + " ET"
print(as_of_date)

3/21/2021 at 8:26 PM ET


Multi-index columna names

In [5]:
bracket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   (Place, Place)                      511 non-null    object
 1   (Mover/Loser, Mover/Loser)          511 non-null    object
 2   (Name Last,First, Name Last,First)  511 non-null    object
 3   (TotalPoints, TotalPoints)          511 non-null    int64 
 4   (PossiblePoints, PossiblePoints)    511 non-null    int64 
 5   (Points Gauge, Points Gauge)        511 non-null    int64 
 6   (PickPct., PickPct.)                511 non-null    object
 7   (Final, 32)                         511 non-null    int64 
 8   (Final, 16)                         511 non-null    int64 
 9   (Final, 8)                          511 non-null    int64 
 10  (Final, 4)                          511 non-null    int64 
 11  (Final, 2)                          511 non-null    int64 

Specify single index for column names

In [6]:
col_names = []
for i, col in enumerate(bracket_df.columns):
    col_name = col[1] if col[0] == col[1] else col[0] + "_" + col[1]
    col_names.append(col_name)
    
bracket_df.columns = col_names

Replace delimiters with `_`

In [7]:
bracket_df.columns = bracket_df.columns.\
    str.replace("/", "_").\
    str.replace(" ", "_").\
    str.replace(",", "_").str.lower()

In [8]:
bracket_df.head()

,place,mover_loser,name_last_first,totalpoints,possiblepoints,points_gauge,pickpct.,final_32,final_16,final_8,final_4,final_2,pickedchamp
0,284t,(-121),"1 Berry, Deiondrea",23,153,23130,61%,21,1,0,0,0,??
1,31t,(+132),"1PEDERSEN,ROBYNALAN",27,163,27136,67%,21,3,0,0,0,??
2,51t,(+23),"2 Berry, Deiondrea",26,152,26126,67%,22,2,0,0,0,??
3,185t,(-111),"2 McNamara, justin",24,162,24138,64%,22,1,0,0,0,??
4,483t,(-35),"2PEDERSEN,ROBYNALAN",19,147,19128,50%,17,1,0,0,0,??


Convert `place` to numeric (strip the `'t'` in ties) and add derived field (`ties`) to indicate how many brackets tied at position.

In [9]:
bracket_df['place'] = bracket_df['place'].str.replace('t', '').astype('int')
bracket_df['ties']= bracket_df.place.map(bracket_df.place.value_counts())

Define a function to standardize player names to group individual players that submitted multiple brackers.

In [10]:
def clean_name(df, col):
    
    # omit digits at start of string
    clean = [re.sub(r'^\d+\s*', r'', x) for x in df[col]]
    
    # omit digits mid-name, keep comma
    clean = [re.sub(r',?\s*,?\d+\s*[,|-]\s*', r', ', x) for x in clean]
    
    # truncate string at first digit
    clean = [re.sub(r'\d.*', r'', x) for x in clean]

    # drop [#-.] from string.
    clean = [re.sub(r'[#\-\.]', r'', x) for x in clean]

    # add space after comma if not present, then drop comma. strip whitespace & lowercase
    clean = [re.sub(r'\s*,\s*', r' ', x).strip(' ').lower() for x in clean]
    
    # convert jr, sr to full suffix (create more "distance" between strings)
    clean = [
        re.sub(r'\s+jr$', r' junior', 
        re.sub(r'\s+sr$', r' senior', x)
        ) for x in clean
    ]
    
    return clean

Apply function, creating `name` field.

In [11]:
bracket_df['name'] = clean_name(bracket_df, 'name_last_first')

Parse out the bracket number associated with each submission.

In [12]:
# filter string on digits only
# prepend 0 to allow int conversion on empty string
bracket_df['bracket_nbr'] = [int('0' + "".join(set(filter(str.isdigit, x)))) for x in bracket_df['name_last_first']]

`pointsgauge` does not have a delimiter, so we correct that here.

In [13]:
# recreate points gauge
bracket_df['points_gauge'] = \
    bracket_df['totalpoints'].astype('str') +\
    ' | ' +\
    (bracket_df['possiblepoints'] - bracket_df['totalpoints']).astype('str')

In [14]:
bracket_df['points_behind'] = bracket_df['totalpoints'].max() - bracket_df['totalpoints']

In [15]:
bracket_df.head()

,place,mover_loser,name_last_first,totalpoints,possiblepoints,points_gauge,pickpct.,final_32,final_16,final_8,final_4,final_2,pickedchamp,ties,name,bracket_nbr,points_behind
0,284,(-121),"1 Berry, Deiondrea",23,153,23 | 130,61%,21,1,0,0,0,??,78,berry deiondrea,1,9
1,31,(+132),"1PEDERSEN,ROBYNALAN",27,163,27 | 136,67%,21,3,0,0,0,??,20,pedersen robynalan,1,5
2,51,(+23),"2 Berry, Deiondrea",26,152,26 | 126,67%,22,2,0,0,0,??,62,berry deiondrea,2,6
3,185,(-111),"2 McNamara, justin",24,162,24 | 138,64%,22,1,0,0,0,??,99,mcnamara justin,2,8
4,483,(-35),"2PEDERSEN,ROBYNALAN",19,147,19 | 128,50%,17,1,0,0,0,??,19,pedersen robynalan,2,13


future analysis to further standardize names (looking for typos, etc.)

In [16]:
# # %pip install python-Levenshtein
# from Levenshtein import distance

# def match(s1, s2):
#     return distance(s1, s2) <= 1

In [17]:
# bracket_df['bracket_nbr'].value_counts().sort_index()

In [18]:
# pd.set_option('display.max_rows',500)
# print(bracket_df['name'].value_counts().sort_index())

In [19]:
bracket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   place            511 non-null    int64 
 1   mover_loser      511 non-null    object
 2   name_last_first  511 non-null    object
 3   totalpoints      511 non-null    int64 
 4   possiblepoints   511 non-null    int64 
 5   points_gauge     511 non-null    object
 6   pickpct.         511 non-null    object
 7   final_32         511 non-null    int64 
 8   final_16         511 non-null    int64 
 9   final_8          511 non-null    int64 
 10  final_4          511 non-null    int64 
 11  final_2          511 non-null    int64 
 12  pickedchamp      511 non-null    object
 13  ties             511 non-null    int64 
 14  name             511 non-null    object
 15  bracket_nbr      511 non-null    int64 
 16  points_behind    511 non-null    int64 
dtypes: int64(11), object(6)
memory usag

In [20]:
bracket_df.groupby(['name']).max().shape

(324, 16)

In [21]:
player_df = \
    bracket_df.groupby(['name']).agg({
        'bracket_nbr': 'count',
        'place': 'min',
        'totalpoints': 'max'
    })

In [22]:
player_df.describe()

,bracket_nbr,place,totalpoints
count,324.000000,324.000000,324.000000
mean,1.577160,200.274691,24.070988
std,0.997009,147.284865,2.523798
min,1.000000,1.000000,17.000000
25%,1.000000,51.000000,23.000000
50%,1.000000,185.000000,24.000000
75%,2.000000,284.000000,26.000000
max,8.000000,509.000000,32.000000


In [23]:
player_bracket_dist = pd.DataFrame(player_df['bracket_nbr'].value_counts())
player_bracket_dist = player_bracket_dist.rename({'bracket_nbr': 'player_cnt'}, axis=1)  # rename

player_bracket_dist['player_pct'] = player_bracket_dist['player_cnt'] / player_bracket_dist['player_cnt'].sum()
player_bracket_dist['bracket_cnt'] = player_bracket_dist['player_cnt'] * player_bracket_dist.index
player_bracket_dist['bracket_pct'] = player_bracket_dist['bracket_cnt'] / player_bracket_dist['bracket_cnt'].sum()

Player-level bracket stats

In [24]:
player_bracket_dist

,player_cnt,player_pct,bracket_cnt,bracket_pct
1,211,0.651235,211,0.412916
2,67,0.206790,134,0.262231
3,29,0.089506,87,0.170254
4,11,0.033951,44,0.086106
5,3,0.009259,15,0.029354
6,2,0.006173,12,0.023483
8,1,0.003086,8,0.015656


Dashboard of the Fiorio's

In [25]:
keep_cols = [
    'name', 'place', 'ties', 'mover_loser', 'totalpoints',
    'points_behind', 'possiblepoints', 'pickpct.', 'pickedchamp'
    ]

In [26]:
pattern = 'fiorio|rasdorf|thomas billy'
fiorio_index = np.where(bracket_df['name'].str.contains(pattern, regex = True))[0]

In [27]:
print(as_of_date)

bracket_df[keep_cols].iloc[fiorio_index, :].\
    sort_values(['totalpoints', 'possiblepoints'], ascending=[False, False]).\
    style.hide_index()

3/21/2021 at 8:26 PM ET


name,place,ties,mover_loser,totalpoints,points_behind,possiblepoints,pickpct.,pickedchamp
fiorio leslie,31,20,(+3),27,5,165,69%,??
fiorio livio,113,72,(+50),25,7,135,64%,??
rasdorf andy,113,72,(+50),25,7,125,64%,??
fiorio marianne,185,99,(+62),24,8,158,61%,??
fiorio luke,452,31,(+31),20,12,158,50%,??
fiorio julia,452,31,(+31),20,12,96,50%,NO
fiorio mara,483,19,(-35),19,13,67,50%,NO
thomas billy,509,2,(-10),17,15,117,44%,??
